# Prueba de concepto de webscraping

In [32]:
import requests, re, json, logging, os, zipfile, sys, datetime, shutil
from pathlib import Path
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from pymongo import MongoClient
import boto3
from botocore.exceptions import ClientError
from typing import Tuple, Optional
from tqdm import tqdm
import numpy as np

In [33]:
# Configuración
ENVIRONMENT = "LOCAL"  # PRODUCTION, LOCAL, SCALEWAY 

# Variables entorno
# Cargar variables de entorno
load_dotenv()

# Variables de la base de datos
MONGODB_INITDB_DATABASE = os.getenv("MONGODB_INITDB_DATABASE")
MONGODB_SERVER_HOST = os.getenv("MONGODB_SERVER_HOST")
MONGODB_SERVER_PORT = os.getenv("MONGODB_SERVER_PORT")
MONGODB_USER = os.getenv("MONGODB_USER")
MONGODB_PASSWORD = os.getenv("MONGODB_PASSWORD")

# Variables de S3
S3_BUCKET = os.getenv("SCALEWAY_BUCKET") if ENVIRONMENT == "SCALEWAY" else os.getenv("MINIO_BUCKET")
S3_ENDPOINT_URL = os.getenv("SCALEWAY_ENDPOINT_URL") if ENVIRONMENT == "SCALEWAY" else os.getenv("MINIO_ENDPOINT_URL")
S3_ACCESS_KEY = os.getenv("SCALEWAY_ACCESS_KEY") if ENVIRONMENT == "SCALEWAY" else os.getenv("MINIO_ACCESS_KEY")
S3_SECRET_KEY = os.getenv("SCALEWAY_SECRET_KEY") if ENVIRONMENT == "SCALEWAY" else os.getenv("MINIO_SECRET_KEY")
S3_REGION = os.getenv("SCALEWAY_REGION") if ENVIRONMENT == "SCALEWAY" else os.getenv("MINIO_REGION")

# Configura el logger
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(funcName)s - %(message)s",
)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Configuracion
DOWNLOAD_PATH = "./downloads"
MONGODB_CONECTION_STRING = f"mongodb://{MONGODB_USER}:{MONGODB_PASSWORD}@{MONGODB_SERVER_HOST}:{MONGODB_SERVER_PORT}/{MONGODB_INITDB_DATABASE}"
PATCHES_PATH = os.path.join(DOWNLOAD_PATH, "patches")
TILE_SIZE = (4096, 4096)
OVER_LAP = 400
PURGE_WHITE_IMAGES = True
S3_BUCKET_IMAGES_PATH = "imagenes"
S3_BUCKET_PATCHES_PATH = "patches"
S3_BUCKET_METADATA_PATH = "metadatos"
# OpenCV
OPENCV_IO_MAX_IMAGE_PIXELS=17179869184 # Para imágenes grandes, ej: barrio3Ombues_20180801_dji_pc_3cm.jpg

# Constantes
URL_MAIN_PAGE = "https://gis.montevideo.gub.uy/pmapper/map.phtml?&config=default&me=548000,6130000,596000,6162000"
URL_TOC = "https://intgis.montevideo.gub.uy/pmapper/incphp/xajax/x_toc.php?"
URL_GENERATE_ZIP = "https://intgis.montevideo.gub.uy/sit/php/common/datos/generar_zip2.php?nom_jpg=/inetpub/wwwroot/sit/mapserv/data/fotos_dron/{id}&tipo=jpg"
URL_DOWNLOAD_ZIP = "https://intgis.montevideo.gub.uy/sit/tmp/{id}.zip"
URL_JS = "https://intgis.montevideo.gub.uy/pmapper/config/default/custom.js"

HEADERS_COMMON = {
    "User-Agent": "Mozilla/5.0",
}

HEADERS_TOC = {
    "User-Agent": "Mozilla/5.0",
    "Referer": URL_MAIN_PAGE,
    "X-Requested-With": "XMLHttpRequest",
    "Content-Type": "application/x-www-form-urlencoded",
}

BODY_TOC = {"dummy": "dummy"}

# Importar OpenCV
import cv2

In [34]:
# Clase singleton para el cliente S3
class S3Client:
    __instance = None

    @staticmethod
    def get_instance():
        if S3Client.__instance is None:
            S3Client()
        return S3Client.__instance

    def __init__(self):
        if S3Client.__instance is not None:
            raise Exception("This class is a singleton!")
        else:
            S3Client.__instance = self
            self.client = boto3.client(
                "s3",
                endpoint_url=S3_ENDPOINT_URL,
                aws_access_key_id=S3_ACCESS_KEY,
                aws_secret_access_key=S3_SECRET_KEY,
                region_name=S3_REGION,
            )

In [35]:
# Clase singleton para MongoDB
class MongoDB:
    __instance = None
    @staticmethod
    def get_instance():
        if MongoDB.__instance is None:
            MongoDB()
        return MongoDB.__instance
    def __init__(self):
        if MongoDB.__instance is not None:
            raise Exception("Esta clase es un singleton!")
        else:
            MongoDB.__instance = self
            self.client = MongoClient(MONGODB_CONECTION_STRING)
            self.db = self.client[MONGODB_INITDB_DATABASE]

In [36]:
def get_url(url: str, headers: dict) -> str:
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.text
    except requests.exceptions.HTTPError as err:
        raise Exception(f"Error al acceder a {url}. Razón: {err}")

In [37]:
def get_toc_html() -> str:
    """
    Inicia una sesión, realiza una petición GET a la página principal y posteriormente
    una petición POST para obtener el HTML que contiene la Table of Contents (TOC).

    Returns:
        str: El contenido HTML que representa la estructura TOC de la página.

    Raises:
        Exception: Si se produce un error HTTP (4xx o 5xx) al acceder a la URL de TOC,
                   se lanza una excepción con un mensaje indicando la razón del fallo.
    """
    
    session = requests.Session()

    try:
        session.get(URL_MAIN_PAGE, headers=HEADERS_COMMON)
        response = session.post(URL_TOC, headers=HEADERS_TOC, data=BODY_TOC)
        response.raise_for_status()
        return response.text
    except requests.exceptions.HTTPError as err:
        raise Exception(f"Error al acceder a {URL_TOC}. Razón: {err}")
    finally:
        session.close()

In [38]:
def get_pictures_list_html(html: str) -> str:
    """
    A partir del HTML recibido, localiza el elemento `<li>` con id 'ligrp_grillaFotosDron'
    y retorna el contenido HTML del `<ul>` padre que contiene las imágenes de drones.

    Args:
        html (str): Cadena que representa el contenido HTML de la página.

    Returns:
        str: El contenido HTML del elemento `<ul>` que contiene las imágenes de drones.

    Raises:
        ValueError: Si no se encuentra el `<li>` con el id especificado o si no existe
                    un `<ul>` padre que lo contenga.
    """
    
    li_grilla_fotos_dron = "ligrp_grillaFotosDron"
    soup = BeautifulSoup(html, 'html.parser')
    
    # 1) Buscamos el <li> deseado
    li_drones_pictures = soup.find("li", {"id": li_grilla_fotos_dron})
    if li_drones_pictures is None:
        raise ValueError("No se encontró el <li> con id='{li_grilla_fotos_dron}' en el HTML.")
    
    # 2) Obtenemos el <ul> que lo contiene
    ul_drones_pictures = li_drones_pictures.find_parent("ul")
    if ul_drones_pictures is None:
        raise ValueError("No se encontró un elemento <ul> padre para el <li> con id='{li_grilla_fotos_dron}'.")

    # Devolvemos el HTML del <ul> que contiene las imágenes
    return str(ul_drones_pictures)

In [39]:
def get_pics_dict(html: str) -> dict:
    """
    Procesa el HTML de un `<ul>` para obtener información sobre cada elemento que
    contiene imágenes de drones. Busca spans con IDs que coincidan con la expresión
    regular `.*fotosDron.*` y extrae un identificador, así como el atributo `title`.

    Args:
        html (str): Contenido HTML que representa la lista de imágenes (un `<ul>`).

    Returns:
        list: Lista de diccionarios, donde cada diccionario contiene:
              - "id": Un índice secuencial que identifica el elemento.
              - "js_name": Valor derivado del ID del span (remueve el prefijo 'spxg_').
              - "title": El valor del atributo title del span interno con clase "grp-title".

    Raises:
        (No lanza excepciones explícitas propias, pero podría propagar excepciones de BeautifulSoup.)
    """
    
    soup = BeautifulSoup(html, "html.parser")

    spans_fotos_dron = soup.find_all("span", id=re.compile(r".*fotosDron.*"))

    resultados = []

    for idx, sp in enumerate(spans_fotos_dron, start=1):
        # 1) Obtener el valor a partir del id del span
        span_id = sp.get("id", "")
        value_attr = span_id.replace("spxg_", "")
        
        # 2) El <span class="grp-title" title="..."> que está dentro
        span_child_element = sp.find("span")
        if not span_child_element:
            # Si no lo encuentra, pasa al siguiente
            logger.warning(f"No se encontró <span class='grp-title'> dentro del span con id='{span_id}'.")
            continue
        title_attr = span_child_element.get("title")
        
        # 3) Construir el objeto según el formato deseado
        resultados.append({
            "id": idx,
            "js_name": value_attr,
            "title": title_attr,
        })

    return resultados

In [40]:
def download_picture(id: str) -> str|None:
    """
    Dada una cadena `id`, realiza un get para generar un archivo ZIP en el servidor y luego
    lo descarga si existe. El ZIP contiene las imágenes correspondientes al identificador.

    Args:
        id (str): Identificador que se usará para generar y descargar el ZIP.

    Returns:
        str | None: La ruta absoluta del archivo ZIP descargado. Retorna None si no se
                    puede generar o descargar el ZIP.

    Raises:
        (Las excepciones de requests se manejan internamente, produciendo logs de error
         o warning en caso de fallos. No se relanza la excepción.)
    """

    # Ruta para generar el archivo ZIP en el servidor
    url_generate_zip = URL_GENERATE_ZIP.format(id=id)

    # Ruta para descargar el archivo ZIP
    url_download_zip = URL_DOWNLOAD_ZIP.format(id=id)
    
    try:
        response = requests.get(url_generate_zip, headers=HEADERS_COMMON)
        response.raise_for_status()
        logger.debug(f"Archivo ZIP generado correctamente en el servidor. URL: {url_generate_zip}")
    except requests.exceptions.RequestException as e:
        logger.warning(f"No se pudo generar el ZIP. URL: {url_generate_zip} - {e}")
    finally:
        try:
            response = requests.get(url_download_zip, headers=HEADERS_COMMON)
            response.raise_for_status()
            logger.debug(f"Archivo ZIP descargado correctamente. URL: {url_download_zip}")

            if response.status_code == 200:
                file_name = f"{id}.zip"
                os.makedirs(DOWNLOAD_PATH, exist_ok=True)

                file_path = os.path.join(DOWNLOAD_PATH, file_name)
                
                with open(file_path, "wb") as f:
                    f.write(response.content)
                logger.debug(f"Archivo '{file_name}' descargado correctamente en {file_path}")
                
                return file_path
        except requests.exceptions.RequestException as e:
            logger.error(f"No se pudo descargar el ZIP. URL: {url_download_zip} - {e}")
            return None

In [41]:
def get_js_as_text() -> str:
    return get_url(URL_JS, HEADERS_COMMON)

In [42]:
def parse_js_cases(js_code: str) -> dict:
    """
    Parsea el código JavaScript para encontrar las líneas que contengan casos y sus file_descarga.
    
    Args:
        js_code (str): El contenido del archivo JavaScript.
    Returns:
        dict: Diccionario con los casos y sus rutas de archivo correspondientes.
    """
    pattern = r"""case\s+'([^']+)':\s*
                  (?:[^\n]*\n)?          # Captura opcional cualquier cosa hasta el fin de línea
                  \s*file_descarga\s*=\s*'([^']+)'\s*;"""
    
    # re.VERBOSE permite escribir la regex más legible con comentarios
    # re.MULTILINE permite que ^ y $ coincidan con principio y fin de línea
    # re.DOTALL hace que . coincida también con saltos de línea
    matches = re.findall(pattern, js_code, re.VERBOSE | re.MULTILINE | re.DOTALL)
    
    return {case_val: file_descarga for case_val, file_descarga in matches}

In [43]:
def get_download_file_id(js_text) -> str:
    # Si tiene el prefijo "fotos_dron/", se lo removemos
    return js_text.removeprefix("fotos_dron/")

In [44]:
def add_file_download_id(resultados: list, mapping: dict) -> list:
    """
    Agrega la clave 'file_download_id' a cada elemento de la lista `resultados`,
    asociando su valor al correspondiente valor de `mapping` si existe.

    Args:
        resultados (list): Lista de diccionarios con al menos la clave "js_name".
        mapping (dict): Diccionario con la relación "js_name" -> "file_descarga".

    Returns:
        list: La lista de diccionarios `resultados` modificada, donde cada elemento
              incluye la clave 'file_download_id' extraída mediante la función
              `obtener_id_file_descarga` si existe, o None en caso contrario.
    """
    
    for item in resultados:
        value_attr = item["js_name"]
        # Si existe la key en el mapping, se la asignamos
        if value_attr in mapping:
            desc = mapping[value_attr]
            item["file_download_id"] = get_download_file_id(desc)
            logger.debug(f"Se asignó el valor '{item['file_download_id']}' a 'file_download_id' para '{value_attr}'.")
        else:
            # Si no está, ponle None o algún valor por defecto
            item["file_download_id"] = None
            logger.warning(f"No se encontró un valor para '{value_attr}' en el mapping. Se asignó None.")
    return resultados

In [45]:
def add_or_sync_downloaded(results: list) -> list:
    """
    Agrega la clave 'downloaded' (estado) a cada elemento de la lista `resultados`, chequeando en la base de datos, indicando si el archivo ZIP asociado a ese elemento se está descargado o no.

    Args:
        resultados (list): Lista de diccionarios con al menos la clave "id".

    Returns:
        list: La lista de diccionarios `resultados` modificada, donde cada elemento
              incluye la clave 'downloaded' con un valor booleano que indica si el archivo
              ZIP se descargó o no.
    """
    imagenes_db = MongoDB.get_instance().db.imagenes
    for item in results:
        # Busco en la base de datos si existe el item.
        # Si existe el item, obtengo el estado.
        # Sino, lo agrego a la base de datos y le pongo el estado en False.
        item_db = imagenes_db.find_one({"id": item["file_download_id"]})
        if item_db:
            item["downloaded"] = item_db["downloaded"]
            logger.info(f"El archivo ZIP asociado a '{item['file_download_id']}' está {'descargado' if item['downloaded'] else 'pendiente'}.")
        else:
            logger.info(f"El archivo ZIP asociado a '{item['file_download_id']}' no está en la base de datos. Se agregará.")
            imagenes_db.insert_one(
                {
                    "id": item["file_download_id"],
                    "js_name": item["js_name"],
                    "title": item["title"],
                    "downloaded": False
                }
            )
            item["downloaded"] = False        
    return results

In [46]:
def extract_files(zip_path: str, id: str = None) -> Tuple[str | None, str | None]:
    """
    Extrae los archivos JPG y JGW de un archivo ZIP descargado.

    Args:
        zip_path (str): Ruta absoluta del archivo ZIP descargado.

    Returns:
        Tuple[str, str]: Tupla con las rutas absolutas de los archivos JPG y JGW extraídos.
                         Si no se pudo extraer alguno de los archivos, se devuelve None.

    Raises:
        Exception: Si el ZIP no contiene exactamente dos archivos, se lanza una excepción.
    """
    extract_dir = os.path.join(DOWNLOAD_PATH, "extracted")
    os.makedirs(extract_dir, exist_ok=True)

    try:
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(extract_dir)

        # Encontrar los archivos JPG y JGW extraídos
        jpg_file = None
        jgw_file = None

        # Obtengo los archivos extraídos
        jpg_file = os.path.join(extract_dir, f"{id}.jpg")
        jgw_file = os.path.join(extract_dir, f"{id}.jgw")

        # Chequeo que existan los archivos
        if not os.path.exists(jpg_file) or not os.path.exists(jgw_file):
            logger.error(f"No se encontraron los archivos JPG y JGW extraídos para {id}.")
            return None, None

        logger.debug(f"Archivo ZIP extraído de {id} correctamente. JPG: {jpg_file}, JGW: {jgw_file}")
        return jpg_file, jgw_file

    except Exception as e:
        logger.error(f"Error extracting files for {id}: {str(e)}")
        return None, None

In [47]:
def upload_img_to_s3(jpg_path: str, jgw_path: str, id: str) -> bool:
    """
    Sube los archivos a Scaleway Object Storage usando boto3.
    """
    try:
        # Get S3 client
        s3_client = S3Client.get_instance().client
        
        # Upload JPG file
        with open(jpg_path, 'rb') as jpg_file:
            s3_client.upload_fileobj(
                jpg_file,
                S3_BUCKET,
                f"{S3_BUCKET_IMAGES_PATH}/{id}.jpg"
            )
            
        # Upload JGW file
        with open(jgw_path, 'rb') as jgw_file:
            s3_client.upload_fileobj(
                jgw_file,
                S3_BUCKET,
                f"{S3_BUCKET_METADATA_PATH}/{id}.jgw"
            )

        logger.debug(f"Files uploaded to Scaleway: {id}")            
        return True
        
    except ClientError as e:
        logger.error(f"Error uploading to Scaleway: {str(e)}")
        return False

In [48]:
def read_jgw_data(jgw_path: str) -> dict:
    """
    Lee un archivo JGW y devuelve un diccionario con los datos.
    """
    try:
        with open(jgw_path, "r") as jgw_file:
            lines = jgw_file.readlines()
            data = {
                "x_pixel_size": float(lines[0].strip()),
                "y_rotation": float(lines[1].strip()),
                "x_rotation": float(lines[2].strip()),
                "y_pixel_size": float(lines[3].strip()),
                "x_origin": float(lines[4].strip()),
                "y_origin": float(lines[5].strip())
            }
            logger.debug(f"JGW file read successfully: {jgw_path}")
            return data
    except Exception as e:
        logger.error(f"Error reading JGW file: {str(e)}")
        return False

In [49]:
def update_mongodb(id: str, jgw_data: dict) -> bool:
    """
    Agrega metadatos de la imagen a MongoDB y actualiza el estado de descarga a True.
    """
    try:
        imagenes_db = MongoDB.get_instance().db.imagenes
        result = imagenes_db.update_one(
            {"id": id},
            {"$set": {
                "downloaded": True,
                "jgw_data": jgw_data,
                "downloaded_date": datetime.datetime.now()
            }}
        )
        if result.modified_count > 0:
            logger.debug(f"MongoDB updated: {id}")
            return True
        else:
            logger.warning(f"MongoDB not updated: {id}")
            return True
    except Exception as e:
        logger.error(f"Error updating MongoDB: {str(e)}")
        return False

In [50]:
def clean_up_directory(dir_path: str) -> None:
    """
    Elimina todo el contenido dentro del directorio especificado.
    Si ocurre algún error, lanza la excepción correspondiente.
    
    Args:
        dir_path (str): Ruta al directorio que se desea limpiar
        
    Returns:
        None
            
    Raises:
        NotADirectoryError: Si la ruta existe pero no es un directorio
        PermissionError: Si no hay permisos suficientes
        OSError: Para otros errores del sistema operativo
    """
    # Convertir a Path para mejor manejo
    path = Path(dir_path)
    
    # Verificar si el directorio existe
    if not path.exists():
        logging.info(f"El directorio {dir_path} no existe. No se requiere limpieza.")
        return
        
    # Verificar si es un directorio
    if not path.is_dir():
        raise NotADirectoryError(f"La ruta {dir_path} no es un directorio")
        
    # Verificar permisos
    if not os.access(path, os.W_OK):
        raise PermissionError(f"Sin permisos de escritura en {dir_path}")

    # Eliminar contenido
    for item in path.iterdir():
        if item.is_file():
            item.unlink()
        elif item.is_dir():
            shutil.rmtree(item)

    logging.debug(f"Directorio {dir_path} limpiado exitosamente")

In [51]:
def cleanup_files(*file_paths: str) -> None:
    """
    Removes temporary files after processing.
    """
    for file_path in file_paths:
        try:
            if file_path and os.path.exists(file_path):
                os.remove(file_path)
        except Exception as e:
            logger.warning(f"Error cleaning up file {file_path}: {str(e)}")

In [52]:
def is_white_image(image, threshold_percent=50, white_threshold=250):
    """
    Detecta imágenes blancas basándose en un porcentaje de píxeles blancos.
    
    Args:
        image: Imagen en formato BGR (OpenCV)
        threshold_percent: Porcentaje de blanco requerido
        white_threshold: Valor mínimo para considerar un pixel como blanco (0-255)
    
    Returns:
        bool: True si la imagen es considerada blanca
        float: Porcentaje de blanco en la imagen
    """
    # Convertir a HSV para mejor manejo del brillo
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Criterios para considerar un pixel como blanco
    white_mask = cv2.inRange(hsv, 
                            np.array([0, 0, white_threshold]),  # Mínimo HSV
                            np.array([180, 30, 255]))  # Máximo HSV

    total_pixels = image.shape[0] * image.shape[1]
    white_pixel_count = cv2.countNonZero(white_mask)
    white_percentage = (white_pixel_count / total_pixels) * 100

    return white_percentage > threshold_percent, white_percentage

In [53]:
def split_image_with_overlap(id: str, jpg_path: str, tile_size: Tuple[int, int], overlap: int, purge_white_images: bool) -> Tuple[list, str]:
    # Cargar la imagen
    image = cv2.imread(jpg_path)
    height, width = image.shape[:2]

    # Crear el directorio de salida si no existe
    output_dir = os.path.join(PATCHES_PATH, id)
    os.makedirs(output_dir, exist_ok=True)

    metadata = []
    patch_id = 0

    # Recorrer la imagen en bloques con superposición
    for y in range(0, height, tile_size[1] - overlap):
        for x in range(0, width, tile_size[0] - overlap):
            # Definir las coordenadas del recorte
            x_end = min(x + tile_size[0], width)
            y_end = min(y + tile_size[1], height)

            # Recortar la región
            tile = image[y:y_end, x:x_end]

            tile_name = f"{id}_patch_{patch_id}.jpg"
            
            # Chequear si la mayoría de la imagen es blanca.
            img_is_white = is_white_image(tile, threshold_percent=60, white_threshold=200)[0]

            # Si la mayoría es balnca, y está activada la opción, no guardar la imagen.
            if purge_white_images:
                if not img_is_white:
                    cv2.imwrite(os.path.join(output_dir, tile_name), tile)
            else:
                cv2.imwrite(os.path.join(output_dir, tile_name), tile)            

            # Guardar los metadatos
            metadata.append({
                "patch_id": patch_id,
                "patch_name": tile_name,
                "x_start": x,
                "y_start": y,
                "x_end": x_end,
                "y_end": y_end,
                "width": x_end - x,
                "height": y_end - y,
                "is_white": img_is_white
            })

            patch_id += 1

    return metadata, output_dir

In [54]:
def upload_patches_to_s3(id: str, patches_dir: str) -> bool:
    """
    Sube los parches a Scaleway Object Storage usando boto3.
    """
    try:
        # Get S3 client
        s3_client = S3Client.get_instance().client

        # Listar los archivos en el directorio
        for patch in os.listdir(patches_dir):
            patch_path = os.path.join(patches_dir, patch)
            with open(patch_path, "rb") as patch_file:
                s3_client.upload_fileobj(
                    patch_file, S3_BUCKET, f"{S3_BUCKET_PATCHES_PATH}/{id}/{patch}"
                )

        logger.debug(f"Patches uploaded to Scaleway: {id}")
        return True

    except ClientError as e:
        logger.error(f"Error uploading patches to Scaleway: {str(e)}")
        return False

In [55]:
def update_mongodb_patches(id: str, patches: list) -> bool:
    """
    Agrega metadatos de los parches a MongoDB.
    """
    try:
        imagenes_db = MongoDB.get_instance().db.imagenes
        result = imagenes_db.update_one(
            {"id": id},
            {"$set": {
                "patches": patches,
                "patches_uploaded": True
            }}
        )
        if result.modified_count > 0:
            logger.debug(f"MongoDB updated with patches: {id}")
            return True
        else:
            logger.warning(f"MongoDB not updated with patches: {id}")
            return True
    except Exception as e:
        logger.error(f"Error updating MongoDB with patches: {str(e)}")
        return False

In [56]:
class ImageProcessingError(Exception):
    """Excepción personalizada para errores en el procesamiento de imágenes"""
    pass


def process_and_upload_image(id: str, patches: bool = False, clean: bool = True) -> bool:
    """
    Downloads, extracts and uploads drone images to Scaleway Object Storage.

    Args:
        download_id (str): The ID of the image to download

    Returns:
        bool: True if processing was successful, False otherwise

    Raises:
        ImageProcessingError: When any step in the process fails
    """
    logger.info(f"Starting processing for download_id: {id}")

    try:
        # Download the ZIP file
        zip_path = download_picture(id)
        if not zip_path:
            raise ImageProcessingError(f"Failed to download ZIP for {id}")
        logger.info(f"ZIP downloaded for {id}")

        # Extract files
        jpg_path, jgw_path = extract_files(zip_path, id)
        if not jpg_path or not jgw_path:
            raise ImageProcessingError(f"Failed to extract files from ZIP for {id}")
        logger.info(f"Files extracted for {id}")

        # Upload to S3
        upload_img_to_s3_result = upload_img_to_s3(jpg_path, jgw_path, id)
        if not upload_img_to_s3_result:
            raise ImageProcessingError(f"Failed to upload files to S3 for {id}")
        logger.info(f"Files uploaded to S3 for {id}")

        jgw_data = read_jgw_data(jgw_path)
        if not jgw_data:
            raise ImageProcessingError(f"Failed to read JGW data for {id}")
        logger.info(f"JGW data read for {id}")

        # Update MongoDB downloaded
        update_mongodb_result = update_mongodb(id, jgw_data)
        if not update_mongodb_result:
            raise ImageProcessingError(f"Failed to update MongoDB downloaded for {id}")
        logger.info(f"MongoDB updated for {id}")

        if patches:
            metadata, output_dir = split_image_with_overlap(
                id,
                jpg_path,
                TILE_SIZE,
                OVER_LAP,
                PURGE_WHITE_IMAGES,
            )
            if not metadata:
                raise ImageProcessingError(f"Failed to split image for {id}")
            logger.info(f"Image split for {id}")

            upload_patches_to_s3_result = upload_patches_to_s3(id, output_dir)
            if not upload_patches_to_s3_result:
                raise ImageProcessingError(f"Failed to upload patches to S3 for {id}")
            logger.info(f"Patches uploaded to S3 for {id}")

            update_mongodb_patches_result = update_mongodb_patches(id, metadata)
            if not update_mongodb_patches_result:
                raise ImageProcessingError(f"Failed to update MongoDB patches for {id}")
            logger.info(f"MongoDB updated with patches for {id}")

        return True

    except ImageProcessingError as e:
        logger.error(str(e))
        return False
    except Exception as e:
        logger.error(f"Unexpected error processing download_id {id}: {str(e)}")
        return False
    finally:
        try:
            if clean:
                clean_up_directory(DOWNLOAD_PATH)
                logger.info(f"Directory cleaned up for {id}")
        except Exception as e:
            logger.error(f"Error cleaning up directory for {id}: {str(e)}")

In [57]:
def process_and_upload_patches(id: str, clean: bool = False) -> bool:
    """Descarga la imagen desde el bucket de S3, la divide en parches y los sube a S3. Finalmente, actualiza 
    la base de datos con los metadatos de los parches.

    Args:
        id (str): ID de la imagen a procesar.
        clean (bool): Indica si se debe limpiar el directorio de descargas al finalizar el procesamiento.

    Returns:
        bool: True si el procesamiento fue exitoso, False en caso contrario.
    """
    logger.info(f"Starting processing for download_id: {id}")

    try:
        # Download the image from S3
        jpg_path = os.path.join(PATCHES_PATH, f"{id}.jpg")

        s3_client = S3Client.get_instance().client
        s3_client.download_file(S3_BUCKET, f"{S3_BUCKET_IMAGES_PATH}/{id}.jpg", jpg_path)
        logger.info(f"Image downloaded from S3 for {id}")

        # Split image into patches
        metadata, output_dir = split_image_with_overlap(
            id,
            jpg_path,
            TILE_SIZE,
            OVER_LAP,
            PURGE_WHITE_IMAGES,
        )
        if not metadata:
            raise ImageProcessingError(f"Failed to split image for {id}")
        logger.info(f"Image split for {id}")

        # Upload patches to S3
        upload_patches_to_s3_result = upload_patches_to_s3(id, output_dir)
        if not upload_patches_to_s3_result:
            raise ImageProcessingError(f"Failed to upload patches to S3 for {id}")
        logger.info(f"Patches uploaded to S3 for {id}")

        # Update MongoDB with patches
        update_mongodb_patches_result = update_mongodb_patches(id, metadata)
        if not update_mongodb_patches_result:
            raise ImageProcessingError(f"Failed to update MongoDB patches for {id}")
        logger.info(f"MongoDB updated with patches for {id}")

        return True

    except ImageProcessingError as e:
        logger.error(str(e))
        return False
    except Exception as e:
        logger.error(f"Unexpected error processing download_id {id}: {str(e)}")
        return False
    finally:
        try:
            if clean:
                clean_up_directory(DOWNLOAD_PATH)
                logger.info(f"Directory cleaned up for {id}")
        except Exception as e:
            logger.error(f"Error cleaning up directory for {id}: {str(e)}")

In [58]:
def get_all_images() -> list:
    """
    Returns all images from MongoDB.
    """
    imagenes_db = MongoDB.get_instance().db.imagenes
    return list(imagenes_db.find())

In [59]:
def set_log_to_file():
    logger = logging.getLogger()
    today = datetime.date.today()

    log_file = f"webscrapping_{today}.log"
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(funcName)s - %(message)s')
    file_handler = logging.FileHandler(log_file)
    file_handler.setFormatter(formatter)

    logger.addHandler(file_handler)

In [60]:
# Imprimir todas las variables de entorno para debug
# logger.info("Variables de entorno:")
# for key, value in os.environ.items():
#     logger.info(f"{key}: {value}")

In [ ]:
# Pasos iniciales
# Chequear conexión a MongoDB
try:
    MongoDB.get_instance()
    # Listar las colecciones
    collections = MongoDB.get_instance().db.list_collection_names()
    logger.info("Connected to MongoDB")
except Exception as e:
    logger.error(f"Error connecting to MongoDB: {str(e)}")
    sys.exit(1)

# Chequear conexión a S3
try:
    S3Client.get_instance()
    # Listar los buckets
    s3_client = S3Client.get_instance().client
    response = s3_client.list_buckets()
    logger.info("Connected to S3")
except Exception as e:
    logger.error(f"Error connecting to S3: {str(e)}")
    sys.exit(1)

# logger.setLevel(logging.INFO)
# results = get_pics_dict(get_pictures_list_html(get_toc_html()))
# mapping = parse_js_cases(get_js_as_text())

# output = add_or_sync_downloaded(add_file_download_id(results, mapping))

# Set log to file
set_log_to_file()
logger.setLevel(logging.INFO)

# Get all images from MongoDB
images = get_all_images()

# Slice the images to test
images = images[0:40]

# Process and upload images with tqdm
logger.info("Starting processing and uploading images")
for image in tqdm(images):
    if image["downloaded"]:
        logger.debug(f"Skipping image {image['id']} as it is already downloaded")
    else:
        logger.info(f"Processing and uploading image {image['id']}")
        process_and_upload_image(image["id"], patches=False, clean=False)
logger.info("Finished processing and uploading images")